In [1]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import gzip
from datetime import datetime, timedelta
from statistics import mean, median
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow
import tensorflow.keras as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization, ReLU, LSTM, Conv1D, Conv2D, Dropout
from tensorflow.keras.activations import sigmoid, tanh
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm
import csv
import random

from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import precision_score as precision
from sklearn.metrics import recall_score as recall
from sklearn.metrics import f1_score as f1

In [2]:
def retrieve_data(varname, filename):
    df = pd.read_csv(filename, index_col=0)
#     df = pd.read_csv(filename)
#     display(df)
    df["Date"] = pd.to_datetime(df["Date"])
    return df

def create_classification_data(df, lookback):
    rows = []

    columns = ['Date', 'SP500_relative_change_perc_1'] # Date and SP500_relative_change_perc_1 from t-0 are added first as target variables 
    
    # create column names based on original with the addition of t-i where i is lookback
    for i in range(1, lookback + 1): # starts at 1 since we do not want t-0 variables apart from 'Date' and 'SP500_relative_change_perc_1'
        new_columns = df.columns.tolist()[1:] # starts at 1 to exclude 'Date' column
        for x in range(len(new_columns)):
            new_columns[x] = new_columns[x] + "_t-" + str(i)
        columns = columns + new_columns
    
    # create lookback data
    for i, row in enumerate(df.iterrows()):
        if i > lookback: # lookback cannot be determined for earlier rows
            new_row = [row[1][0], row[1][1]] # add target 'Date' and 'SP500_relative_change_perc_1 '
            for x in range(1, lookback + 1): # starts at 1 since we do not want t-0 variables apart from 'Date' and 'SP500_relative_change_perc_1'
                add_row = df.iloc[i - x].tolist()[1:] # starts at 1 to exclude 'Date' column
                new_row = new_row + add_row
            rows.append(new_row)
    df2 = pd.DataFrame(rows)
    df2.columns = columns
    return df2

def create_train_val_test(df, year_val, year_test, perc_train=None):
    if perc_train == None:
        # assumes years_train < year_val < year_test
        df["Date"] = pd.to_datetime(df["Date"])
        
        val = df[df['Date'].dt.year == year_val]
        test = df[df['Date'].dt.year == year_test]
        train = df[df['Date'].dt.year < year_val]
    else:
        train = df.head(round(len(df) * perc_train))
        val = df.tail(len(df) - len(train))
        test = val.tail(round(0.5 * len(val)))
        val = val.head(len(val) - len(test))
    y_train = train['SP500_relative_change_perc_1']
    x_train = train.drop(['SP500_relative_change_perc_1'], axis=1)
    
    y_val = val['SP500_relative_change_perc_1']
    x_val = val.drop(['SP500_relative_change_perc_1'], axis=1)
    
    y_test = test['SP500_relative_change_perc_1']
    x_test = test.drop(['SP500_relative_change_perc_1'], axis=1)
    
    return x_train, y_train, x_val, y_val, x_test, y_test

def scale_data(x):
    standard_scaler = MinMaxScaler()
    x_scaled = pd.DataFrame(standard_scaler.fit_transform(x), columns=x.columns)
    return x_scaled

In [3]:
def label_data(y):
    positives = []
    negatives = []
    y = list(y)
    
    labels = []
    for dev in y:
        if dev >= 0:
            labels.append(1)
        else:
            labels.append(0)
    return labels

In [4]:
class LSTM_model(object):
    def __init__(self, x, activation_functions, batch_sizes, lookbacks): 
        self.activation_function = activation_functions[x[0]]
        
        self.dense1 = x[1]
        self.dense2 = x[2]
        self.dense3 = x[3]

        self.dropout1 = x[4]
        self.dropout2 = x[5]
        
        self.epochs = x[6]
        self.batch_size = batch_sizes[x[7]]
        
        self.lookback = lookbacks[x[8]]
        
        
    def fit(self):
        global df

        val_year = 2018
        test_year = 2019

        if self.lookback == 3:
            features_set = ['SP500_stochastic_D_5_5_t-3', 'SP500_momentum_16_t-1', 'SP500_williams_R_10_t-1', 'SP500_williams_R_20_t-1', 'SP500_AD_MACD_12_26_t-3', 'SP500_stochastic_D_5_5_t-1', 'SP500_stochastic_K_50_t-3', 'SP500_F_relative_change_perc_1_t-3', 'SP500_stochastic_K_5_t-3', 'SP500_williams_R_5_t-1', 'Gold_F_relative_change_perc_1_t-2', 'SP500_stochastic_K_10_t-1', 'SP500_AD_oscillator_t-1', 'SP500_williams_R_10_t-2', 'SP500_momentum_8_t-1', 'SP500_stochastic_D_5_5_t-2']
        elif self.lookback == 5:
            features_set = ['SP500_momentum_16_t-1', 'SP500_williams_R_5_t-5', 'SP500_stochastic_K_50_t-5', 'SP500_williams_R_20_t-1', 'SP500_relative_change_perc_1_t-4', 'SP500_williams_R_50_t-3', 'SP500_AD_oscillator_t-4', 'SP500_stochastic_K_5_t-4', 'SP500_F_relative_change_perc_1_t-3', 'SP500_AD_oscillator_t-3', 'Silver_F_relative_change_perc_1_t-5', 'Gold_F_relative_change_perc_1_t-2', 'SP500_stochastic_K_50_t-4', 'SP500_williams_R_50_t-4', 'SP500_stochastic_K_10_t-1', 'Silver_F_relative_change_perc_1_t-4', 'SP500_AD_oscillator_t-1', 'SP500_williams_R_10_t-2', 'SP500_stochastic_K_5_t-2', 'SP500_stochastic_D_5_5_t-2', 'SP500_williams_R_20_t-3', 'Copper_F_relative_change_perc_1_t-4']
        elif self.lookback == 10:
            features_set = ['SP500_stochastic_K_50_t-5', 'SP500_williams_R_10_t-3', 'SP500_relative_change_perc_1_t-4', 'SP500_williams_R_50_t-3', 'Copper_F_relative_change_perc_1_t-7', 'SP500_stochastic_K_20_t-4', 'Gold_F_relative_change_perc_1_t-10', 'SP500_stochastic_K_5_t-4', 'SP500_momentum_16_t-7', 'SP500_stochastic_K_10_t-10', 'Silver_F_relative_change_perc_1_t-5', 'Gold_F_relative_change_perc_1_t-2', 'SP500_stochastic_K_50_t-4', 'SP500_williams_R_50_t-4', 'SP500_stochastic_K_10_t-1', 'SP500_stochastic_K_10_t-7', 'SP500_AD_oscillator_t-9', 'SP500_AD_oscillator_t-1', 'SP500_stochastic_K_5_t-2', 'Silver_F_relative_change_perc_1_t-10', 'SP500_stochastic_D_5_5_t-5', 'SP500_williams_R_10_t-7']
        elif self.lookback == 20:
            features_set = ['Silver_F_relative_change_perc_1_t-16', 'Copper_F_relative_change_perc_1_t-19', 'Gold_F_relative_change_perc_1_t-14', 'SP500_stochastic_K_50_t-13', 'Silver_F_relative_change_perc_1_t-5', 'SP500_AD_oscillator_t-6', 'SP500_F_relative_change_perc_1_t-13', 'SP500_AD_oscillator_t-17', 'Silver_F_relative_change_perc_1_t-8', 'SP500_stochastic_K_10_t-1', 'SP500_F_relative_change_perc_1_t-12', 'SP500_relative_change_perc_1_t-18', 'SP500_stochastic_K_5_t-6', 'SP500_stochastic_K_50_t-11', 'SP500_williams_R_5_t-5', 'SP500_williams_R_10_t-8', 'SP500_AD_oscillator_t-4', 'Copper_F_relative_change_perc_1_t-4', 'SP500_stochastic_K_10_t-10', 'SP500_williams_R_50_t-11', 'SP500_williams_R_50_t-4', 'SP500_momentum_8_t-20', 'SP500_momentum_8_t-4', 'SP500_stochastic_K_5_t-20', 'Silver_F_relative_change_perc_1_t-19', 'SP500_F_relative_change_perc_1_t-17', 'SP500_williams_R_5_t-18', 'SP500_F_relative_change_perc_1_t-10', 'SP500_momentum_8_t-14', 'Gold_F_relative_change_perc_1_t-10', 'SP500_stochastic_K_5_t-4', 'SP500_AD_oscillator_t-14', 'SP500_stochastic_K_50_t-4', 'Gold_F_relative_change_perc_1_t-8', 'SP500_stochastic_K_10_t-7', 'Silver_F_relative_change_perc_1_t-4', 'Silver_F_relative_change_perc_1_t-1', 'SP500_AD_oscillator_t-1', 'Copper_F_relative_change_perc_1_t-11', 'SP500_AD_oscillator_t-15', 'SP500_williams_R_10_t-11', 'SP500_stochastic_D_5_5_t-5', 'SP500_momentum_16_t-5', 'SP500_relative_change_perc_1_t-16', 'SP500_stochastic_K_50_t-5', 'SP500_stochastic_K_5_t-16', 'SP500_AD_MACD_12_26_t-8', 'SP500_momentum_16_t-7', 'Copper_F_relative_change_perc_1_t-6', 'SP500_stochastic_D_5_5_t-19', 'SP500_AD_oscillator_t-20', 'SP500_relative_change_perc_1_t-2', 'SP500_williams_R_50_t-14', 'SP500_stochastic_K_50_t-14', 'SP500_momentum_16_t-14', 'SP500_momentum_16_t-20', 'Gold_F_relative_change_perc_1_t-13', 'SP500_williams_R_5_t-6']
        
        features_set.insert(0, 'SP500_relative_change_perc_1')
        features_set.insert(0, 'Date')
        
        df_class = create_classification_data(df, self.lookback)
        df_class = df_class[features_set].copy()
        x_train, y_train, x_val, y_val, x_test, y_test = create_train_val_test(df_class, val_year, test_year)


        y_train = label_data(y_train)
        y_val = label_data(y_val)
        y_test = label_data(y_test)

        train_date = x_train[['Date']]
        x_train = x_train.drop(['Date'], axis=1)
        val_date = x_val[['Date']]
        x_val = x_val.drop(['Date'], axis=1)
        test_date = x_test[['Date']]
        x_test = x_test.drop(['Date'], axis=1)
        
        x_train = scale_data(x_train)
        x_val = scale_data(x_val)
        x_test = scale_data(x_test)

        x_train = np.asarray(x_train)
        x_val = np.asarray(x_val)
        x_test = np.asarray(x_test)
        y_train = np.asarray(y_train)
        y_val = np.asarray(y_val)
        y_test = np.asarray(y_test)

        x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
        x_val = x_val.reshape((x_val.shape[0], 1, x_val.shape[1]))
        x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))

        y_train = y_train.reshape((y_train.shape[0], 1))
        y_val = y_val.reshape((y_val.shape[0], 1))
        y_test = y_test.reshape((y_test.shape[0], 1))

        tensorflow.random.set_seed(111)
        np.random.seed(111)
        random.seed(111)
        
        model = Sequential()
        model.add(Dense(self.dense1, activation=self.activation_function))
        if self.dropout1 > 0:
            model.add(Dropout(self.dropout1))
        if self.dense2 > 0:
            model.add(Dense(self.dense2, activation=self.activation_function))
        if self.dropout2 > 0:
            model.add(Dropout(self.dropout2))
        if self.dense3 > 0:
            model.add(Dense(self.dense3, activation=self.activation_function))
        model.add(Dense(1, activation='sigmoid'))
        
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["acc"])
        history = model.fit(x_train, y_train, epochs=self.epochs, batch_size=self.batch_size, verbose=0, validation_data=(x_val, y_val), shuffle=False)
        
#         val_loss = mean(history.history['val_acc'][-5:])
        val_loss = mean(history.history['val_loss'][-5:])
        return val_loss

In [5]:
def calculate_fitness(val_loss, x):
    fitness = val_loss
    return fitness

In [6]:
class EA(object):
    def __init__(self, population_size, activation_functions, batch_sizes, lookbacks):
        self.population_size = population_size
        self.a = 0.2
        self.activation_functions = activation_functions
        self.batch_sizes = batch_sizes
        self.lookbacks = lookbacks
        
    def evaluate(self, x):
        """
        include in fitness function
            relative difference between train_loss and val_loss (smaller is better)
            number of layers (smaller is better)
            bottleneck size (smaller is better)
            val_loss (smaller is better)
        
        """
        lstm = LSTM_model(x, self.activation_functions, self.batch_sizes, self.lookbacks)
        val_loss = lstm.fit()
        fitness = calculate_fitness(val_loss, x)
        return fitness
    
    def select_triple(self, candidate, population):
        # select three random instances for differential evolution
        x1, x2, x3 = np.random.choice(range(len(population))), np.random.choice(range(len(population))), np.random.choice(range(len(population)))
        while candidate == x1 or candidate == x2 or candidate == x3 or x1 == x2 or x2 == x3 or x1 == x3:
            # keep selecting new ones until candidate != x1 != x2 != x3
            x1, x2, x3 = np.random.choice(range(len(population))), np.random.choice(range(len(population))), np.random.choice(range(len(population)))
        return population[x1], population[x2], population[x3]
    
    def mutate(self, x1, x2, x3):
        mutated = x1 + (self.a * (x3 - x2))
    
        # activation function
        mutated[0] = round(mutated[0])
        mutated[0] = min(mutated[0], len(self.activation_functions) - 1)
        mutated[0] = max(0, mutated[0])
        
        # dense layer 1
        mutated[1] = round(mutated[1])
        mutated[1] = max(1, mutated[1]) # must be at least one

        # dense layer 2
        mutated[2] = round(mutated[2])
        mutated[2] = max(0, mutated[2])

        # dense layer 3
        mutated[3] = round(mutated[3])
        mutated[3] = max(0, mutated[3])
        
        # dropout layer 1
        mutated[4] = max(0.05, mutated[4])
        mutated[4] = min(0.95, mutated[4])
        mutated[4] = round(mutated[4],2)
        prob = np.random.randint(0, 2)
        if prob == 1:
            mutated[4] = round(random.uniform(0.05, 0.2),2)
        
        # dropout layer 2
        mutated[5] = max(0.05, mutated[5])
        mutated[5] = min(0.95, mutated[5])
        mutated[5] = round(mutated[5],2)
        prob = np.random.randint(0, 2)
        if prob == 1:
            mutated[5] = round(random.uniform(0.05, 0.2),2)
        
        # epochs
        mutated[6] = round(mutated[6])
        mutated[6] = max(6, mutated[6])

        # batch size
        mutated[7] = round(mutated[7])
        mutated[7] = min(mutated[7], len(self.batch_sizes) - 1)
        mutated[7] = max(0, mutated[7])
        
        mutated[8] = round(mutated[8])
        mutated[8] = min(mutated[8], len(self.lookbacks) - 1)
        mutated[8] = max(0, mutated[8])

        return mutated
        
    def recombine(self, candidate, mutation):
        for i in range(candidate.shape[0]):
            prob = np.random.randint(0, 2)
            if prob == 1:
                candidate[i] = mutation[i]
        return candidate

    def select(self, x_new, f_new, x_old, f_old):
        x_cat = np.concatenate([x_new, x_old], 0)
        f_cat = np.concatenate([f_new, f_old])
        ind = np.argsort(f_cat)
        x = x_cat[ind]
        f = f_cat[ind]
#         return x[-self.population_size:], f[-self.population_size:]
        return x[:self.population_size], f[:self.population_size]
    
    def step(self, x_old, f_old):
        x = np.copy(x_old)
        f = np.copy(f_old)
        for i in tqdm(range(self.population_size), total=self.population_size):
            # choose candidate
            candidate = x[i]
            # select 3 instances for differential evolution
            x1, x2, x3 = self.select_triple(i, x)
            # mutate 3 instances
            mutated_triple = self.mutate(x1, x2, x3)
            # recombine candidate with mutation
            candidate = self.recombine(candidate, mutated_triple)
            x[i] = candidate
            # evaluate candidate solution
            f_candidate = self.evaluate(candidate)
            f[i] = f_candidate
        # select survivors
        x, f = self.select(x, f, x_old, f_old)
        return x, f

In [7]:
def init_population(population_size, activation_functions, batch_sizes, lookbacks):
    # generate initial population
    population = []
    print("Creating initial population...")
    for i in tqdm(range(population_size), total=population_size):
        activation_function = random.randint(0, len(activation_functions) - 1)
        
        lookback = random.randint(0, len(lookbacks) - 1)
        
        variables = lookbacks[lookback] * 18
        
        dense1 = random.randint(4, 1 * variables)
        dense2 = random.randint(4, 1 * variables)
        dense3 = random.randint(4, 1 * variables)
        
        dropout1 = round(random.uniform(0.05, 0.95),2)
        dropout2 = round(random.uniform(0.05, 0.95),2)

        epochs = random.randint(10, 500)
        
        batch_size = random.randint(0, len(batch_sizes) - 1)
        

        population.append(np.asarray([activation_function, dense1, dense2, dense3, dropout1, dropout2, epochs, batch_size, lookback], dtype='object'))
    print("Initial population ready")
    return np.asarray(population)

def evaluate_init_population(ea, x):
    # evaluate initial population
    f = []
    print("Evaluating initial population...")
    for i in tqdm(range(x.shape[0]), total=x.shape[0]):
        instance = x[i]
        f.append(ea.evaluate(instance))
    print("Evaluation initial population completed")
    return np.asarray(f)

def print_best(x, activation_functions, batch_sizes, fitness, features):
    print(f"\nMost suitable parameters -- Loss of {fitness}:")
    print(f"\tActivation function:           \t{activation_functions[x[0]]}")
    print(f"\tDense nodes layer 1:           \t{x[1]}")
    print(f"\tDense nodes layer 2:           \t{x[2]}")
    print(f"\tDense nodes layer 1:           \t{x[3]}")
    print(f"\tDropout layer 1:               \t{x[4]}")
    print(f"\tDropout layer 2:               \t{x[5]}")
    print(f"\tEpochs trained:                \t{x[6]}")
    print(f"\tBatch Size:                    \t{batch_sizes[x[7]]}")
    print(f"\tLookback:                      \t{lookbacks[x[8]]}")

def plot_convergence(f_best):
    fig1 = make_subplots(rows=1, cols=1, specs=[[{'type':'xy'}]])
    
    x_values = []
    for i in range(len(f_best)):
        x_values.append(i)
    fig1.add_trace(go.Scatter(x=x_values, y=f_best, mode="lines"), row=1, col=1)

    fig1.update_layout(
        title = f'Validation Accuracy Over NN Tuning Generations', 
        xaxis1 = dict(title_text = 'Generation'),
        yaxis1 = dict(title_text = "Validation Accuracy")
    )
    fig1.write_image("Plots/opt nn-3 sp500 binary direction.png")
    fig1.show()

def validate_best(x, ea):
    print("\nValidating solution...")
    ea.evaluate(x)
    print("Solution validated")

In [8]:
df = retrieve_data("SP500", "Dataset v3/SP500_reduced_data_20220425.csv")

population_size = 80
generations = 10
lookbacks = [3, 5, 10, 20]
activation_functions = ['sigmoid', 'tanh', 'relu']
batch_sizes = [64, 128, 256]

ea = EA(population_size, activation_functions, batch_sizes, lookbacks)
x = init_population(population_size, activation_functions, batch_sizes, lookbacks)
print(x)
f = evaluate_init_population(ea, x)

populations = []
populations.append(x)
# f_best = [f.max()]
f_best = [f.min()]

start_time = datetime.now()

print("--> STARTING EVOLUTION")
early_stop = 0
for i in range(generations):
#     print(f'Generation: {i}\tBest fitness: {f.max()}')
    print(f'Generation: {i}\tBest fitness: {f.min()}')
    x, f = ea.step(x, f)
    print(x)
    populations.append(x)

#     if f.max() > f_best[-1]:
    if f.min() < f_best[-1]:
#         f_best.append(f.max())
        f_best.append(f.min())
        early_stop = 0
    else:
        f_best.append(f_best[-1])
        early_stop += 1
    if early_stop == 3:
        print("Early stop triggered at generation {i} after not improving fitness for three generations")
        break
print("--> EVOLUTION FINISHED")

end_time = datetime.now()
evolution_time = end_time - start_time
evolution_time_seconds = evolution_time.total_seconds()
print(f"\nElapsed time in minutes: {evolution_time_seconds/60}")

print(f)
print(f.min())
# index_best_parameters = np.where(f == f.max())[0][0]
index_best_parameters = np.where(f == f.min())[0][0]
print(index_best_parameters)
# print_best(x[index_best_parameters], activation_functions, batch_sizes, f.max(), features)
print_best(x[index_best_parameters], activation_functions, batch_sizes, f.min(), lookbacks)
validate_best(x[index_best_parameters], ea)
plot_convergence(f_best)

  0%|                                                                                           | 0/80 [00:00<?, ?it/s]

Creating initial population...
Initial population ready
[[0 340 325 278 0.5 0.59 37 0 3]
 [1 354 115 224 0.88 0.75 334 0 3]
 [1 88 69 9 0.17 0.53 286 0 1]
 [2 51 165 176 0.13 0.26 404 1 2]
 [0 89 58 57 0.7 0.07 144 2 1]
 [0 40 15 238 0.23 0.94 34 1 3]
 [2 9 49 4 0.17 0.9 282 2 0]
 [1 17 53 25 0.55 0.19 240 1 0]
 [0 89 64 57 0.72 0.14 50 1 1]
 [1 15 161 90 0.23 0.94 478 0 2]
 [2 41 51 13 0.89 0.6 273 1 0]
 [0 81 125 129 0.19 0.13 114 2 2]
 [1 32 34 82 0.14 0.85 185 2 2]
 [1 227 260 6 0.61 0.7 341 1 3]
 [0 8 25 35 0.59 0.74 248 1 0]
 [0 285 49 33 0.31 0.58 208 0 3]
 [1 84 76 54 0.94 0.48 313 1 1]
 [1 42 25 35 0.31 0.43 16 0 0]
 [0 208 174 190 0.39 0.28 451 1 3]
 [0 51 7 43 0.57 0.33 401 1 1]
 [0 20 15 36 0.88 0.86 328 2 0]
 [2 49 27 36 0.33 0.77 59 1 0]
 [0 19 8 27 0.55 0.12 304 1 0]
 [0 4 74 78 0.1 0.21 433 2 2]
 [0 80 120 90 0.78 0.53 386 1 2]
 [0 13 11 18 0.15 0.12 78 1 0]
 [1 93 21 101 0.38 0.13 128 0 2]
 [1 126 45 31 0.8 0.81 422 1 2]
 [0 126 6 124 0.66 0.68 445 0 2]
 [1 27 28 14 0.

2022-05-18 14:22:25.544687: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-18 14:22:25.550862: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-18 14:22:25.752528: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
  0%|                                                                                           | 0/80 [00:00<?, ?it/s]

Evaluation initial population completed
--> STARTING EVOLUTION
Generation: 0	Best fitness: 0.6933348298072814


  0%|                                                                                           | 0/80 [00:00<?, ?it/s]

...

  0%|                                                                                           | 0/80 [00:00<?, ?it/s]

[[0 126 6 124 0.66 0.68 445 0 2]
 [1 354 115 224 0.88 0.75 334 0 3]
 [0 112 85 199 0.93 0.46 266 0 3]
 [0 40 15 238 0.23 0.94 34 1 3]
 [0 340 325 278 0.5 0.59 37 0 3]
 [0 1 93 8 0.56 0.65 76 1 2]
 [0 64 85 36 0.51 0.78 89 0 1]
 [0 27 4 47 0.41 0.91 81 0 0]
 [0 182 39 6 0.27 0.89 118 0 3]
 [2 299 252 337 0.94 0.71 19 0 3]
 [0 89 64 57 0.72 0.14 50 1 1]
 [0 125 246 354 0.12 0.69 55 2 3]
 [2 49 27 36 0.33 0.77 59 1 0]
 [0 19 6 0 0.93 0.46 340 0 3]
 [0 1 0 0 0.5 0.59 385 0 3]
 [2 1 0 0 0.79 0.62 385 0 3]
 [0 86 93 8 0.56 0.65 76 0 2]
 [0 147 125 129 0.19 0.13 114 1 2]
 [1 1 0 0 0.88 0.75 385 0 3]
 [0 8 25 35 0.59 0.74 248 1 0]
 [0 1 0 0 0.27 0.89 385 0 3]
 [0 1 0 0 0.27 0.89 385 0 3]
 [0 6 42 33 0.86 0.28 435 1 0]
 [0 19 6 0 0.27 0.89 340 0 3]
 [0 1 0 0 0.93 0.46 385 0 3]
 [0 1 0 0 0.93 0.46 385 0 3]
 [2 2 6 0 0.79 0.62 340 0 3]
 [2 1 0 0 0.94 0.71 385 0 3]
 [0 6 6 0 0.53 0.45 340 0 3]
 [2 2 6 0 0.94 0.71 340 0 3]
 [0 80 120 90 0.78 0.53 386 1 2]
 [2 93 0 0 0.94 0.71 385 0 3]
 [1 2 6 0 0.7

100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [48:26<00:00, 36.33s/it]

[[0 126 6 124 0.66 0.68 445 0 2]
 [1 354 115 224 0.88 0.75 334 0 3]
 [0 112 85 199 0.93 0.46 266 0 3]
 [0 40 15 238 0.23 0.94 34 1 3]
 [0 340 325 278 0.5 0.59 37 0 3]
 [0 1 93 8 0.56 0.65 76 1 2]
 [0 64 85 36 0.51 0.78 89 0 1]
 [0 27 4 47 0.41 0.91 81 0 0]
 [0 182 39 6 0.27 0.89 118 0 3]
 [0 4 23 31 0.51 0.78 309 0 1]
 [0 4 23 31 0.51 0.78 309 0 1]
 [0 4 23 31 0.23 0.94 309 1 3]
 [0 4 23 31 0.23 0.94 309 1 3]
 [0 4 23 31 0.23 0.94 309 1 3]
 [0 6 23 31 0.93 0.46 309 0 3]
 [0 6 23 31 0.93 0.46 309 0 3]
 [0 4 23 31 0.41 0.91 309 0 0]
 [0 4 23 31 0.41 0.91 309 0 0]
 [0 4 23 31 0.41 0.91 309 0 0]
 [0 3 23 31 0.93 0.46 309 0 3]
 [0 3 23 31 0.93 0.46 309 0 3]
 [0 6 23 31 0.51 0.78 309 0 1]
 [0 3 23 31 0.27 0.89 309 0 3]
 [0 3 23 31 0.27 0.89 309 0 3]
 [0 3 23 31 0.27 0.89 309 0 3]
 [0 6 23 31 0.41 0.91 309 0 0]
 [0 6 23 31 0.56 0.65 309 0 2]
 [0 6 23 31 0.66 0.68 309 0 2]
 [0 6 23 31 0.27 0.89 309 0 3]
 [0 4 23 31 0.66 0.68 309 0 2]
 [0 4 23 31 0.66 0.68 309 0 2]
 [0 6 23 31 0.5 0.59 309 0 3]

NameError: name 'features' is not defined

In [10]:
index_best_parameters = np.where(f == f.min())[0][0]
print(index_best_parameters)
print_best(x[index_best_parameters], activation_functions, batch_sizes, f.max(), lookbacks)
validate_best(x[index_best_parameters], ea)
plot_convergence(f_best)

0

Most suitable parameters -- Loss of 0.6961323499679566:
	Activation function:           	sigmoid
	Dense nodes layer 1:           	126
	Dense nodes layer 2:           	6
	Dense nodes layer 1:           	124
	Dropout layer 1:               	0.66
	Dropout layer 2:               	0.68
	Epochs trained:                	445
	Batch Size:                    	64
	Lookback:                      	10

Validating solution...
Solution validated
